# Ноутбук как пример взаимодействия с АПИ LLM и VLM моделей Yandex Cloud'а

## Imports

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import sys

sys.path.append(os.path.dirname(os.getcwd()))
# sys.path

In [ ]:
import pandas as pd

In [ ]:
# from market_oracle_lib import hello
# from market_oracle_lib.data import t_bank, yfinance, apimoex
# from market_oracle_lib.data.data_funcs import (
#     create_data_loaders,
#     create_data_frames,
#     create_data_sets,
#     create_base_data_frames,
# )
# from market_oracle_lib.consts import RU_DEFAULT_SYMBOLS

In [ ]:
import base64
import json
from pathlib import Path

In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()

YA_TOKEN = os.getenv("YA_GPT_TOKEN")
YA_FOLDER_ID = os.getenv("YA_FOLDER_ID")

In [ ]:
print(f"{YA_TOKEN=}")
print(f"{YA_FOLDER_ID=}")

In [ ]:
from yandex_cloud_ml_sdk import YCloudML

sdk = YCloudML(
    folder_id=YA_FOLDER_ID,
    auth=YA_TOKEN,
)
sdk

## Пытаюсь содзать датасет в yandex-cloud

In [ ]:
for dataset in sdk.datasets.list():
    print(f"List of existing datasets {dataset=}")

In [ ]:
with open("./ya_datasets/ex_template.json", "r") as f:
    ya_dataset = json.load(f)
ya_dataset


In [ ]:
# Чтение и кодирование изображения в base64
image_path = Path("./logs/YDEX_price_channel_20250612_133159.png")
with open(image_path, "rb") as image_file:
    encoded_image = base64.b64encode(image_file.read()).decode('utf-8')

# Создание запроса в нужном формате
vision_messages = {
    "request": [{
        "role": "user",
        "content": [
            {
                "type": "text",
                "text": "Опиши, что происходит на графике и дай рекомендации по выставлению ордеров на бирже"
            },
            {
                "type": "image",
                "image": encoded_image
            }
        ]
    }]
}

# Сохранение запроса в файл
with open("./ya_datasets/vlm_request_1.json", "w", encoding="utf-8") as f:
    json.dump(vision_messages, f)

print(json.dumps(vision_messages, indent=4))
vision_messages

In [ ]:
# Создаем датасет
dataset_draft = sdk.datasets.draft_from_path(
    task_type="ImageTextToTextGenerationRequest",
    path="./ya_datasets/vlm_request_1.json",
    upload_format="jsonlines",
    name="vlm_attempt_1",
)

# Дождемся окончания загрузки данных и создания датасета
operation = dataset_draft.upload_deferred()
tuning_dataset = operation.wait()
print(f"new {tuning_dataset=}")

In [ ]:
dir(sdk.models)

In [ ]:
model = sdk.models.completions('qwen2.5-vl-7b-instruct').configure(temperature=0.5, max_tokens=2000)
print(f"{tuning_dataset.id=}")
print(f"{model=}")
# operation = model.run(tuning_dataset.id)

In [ ]:
!pip install yandex-cloud-ml-sdk==0.9.0 -q

In [ ]:
operation = model.batch.run_deferred(tuning_dataset.id)
# operation = model.run_deferred(tuning_dataset.id)


In [ ]:
resulting_dataset = operation.wait()

In [ ]:
res = resulting_dataset.download(download_path="./logs")

In [ ]:
!pip install pyarrow

In [ ]:
for line in resulting_dataset.read():
    print(line['response'])

In [ ]:
# Установите OpenAI SDK с помощью pip
# pip install openai
import openai

YANDEX_CLOUD_FOLDER = "..."
YANDEX_CLOUD_API_KEY = "..."

client = openai.OpenAI(
    api_key=YANDEX_CLOUD_API_KEY,
    base_url="https://llm.api.cloud.yandex.net/v1"
)

client

In [ ]:
response = client.chat.completions.create(
    model=f"gpt://{YANDEX_CLOUD_FOLDER}/yandexgpt/latest",
    messages=[
        {"role": "system", "content": "Ты очень умный ассистент."},
        {"role": "user", "content": "Что умеют большие языковые модели?"}
    ],
    max_tokens=2000,
    temperature=0.3,
    stream=True
)

for chunk in response:
    if chunk.choices[0].delta.content is not None:
        print(chunk.choices[0].delta.content, end="")


In [ ]:
vision_messages

In [ ]:
response = client.chat.completions.create(
    model=f"gpt://{YANDEX_CLOUD_FOLDER}/qwen2.5-vl/latest",
    messages=[vision_messages],
    max_tokens=2000,
    temperature=0.3,
    stream=True
)

for chunk in response:
    if chunk.choices[0].delta.content is not None:
        print(chunk.choices[0].delta.content, end="")


## LLM

In [ ]:
perplex_ans = """<p>Последние новости по компании <b>Норильский никель</b> включают в себя несколько важных событий и решений:</p>

<p>В 2025 году компания объявила об итогах производственной деятельности за первый квартал, что указывает на восстановление после предыдущих проблем, включая пожар в 2022 году[1].</p>

<p>Кроме того, <b>Норникель</b> активно сотрудничает с технологическими компаниями, такими как <b>Yandex B2B Tech</b>, для развития своих проектов[2].</p>

<p>Совет директоров компании рекомендовал не выплачивать дивиденды по итогам прошлого года, что может быть связано с финансовой стратегией и приоритетом инвестиций в восстановление и развитие производства[3].</p>

<p>В секторе добычи и переработки металлов <b>Норникель</b> планирует выпускать 3 тыс. тонн кобальта ежегодно, начиная с 2025 года, что является значительным шагом в восстановлении после пожара на одном из предприятий[5].</p>

<p><i>Семантический анализ</i> показывает, что компания переживает период восстановления и развития, с акцентом на технологических инновациях и расширении производства. Однако решение о не выплате дивидендов может быть воспринято как временная мера для обеспечения стабильности и инвестиций в будущее.</p>

<p>В целом, <b>фон компании</b> можно оценить как <b>нейтральный</b>, поскольку она переживает период восстановления и развития, но также сталкивается с финансовыми вызовами.</p>"""
perplex_ans

In [ ]:
system_prompt = """Ты умный ассистент, которому нужно поправить ответ, который дал LLM.
Ответ является текстом с html-тегами, который потом будет отправлен в телеграм.
Ты должен исправить все ошибки в тексте, которые могут быть связаны с html-тегами.
Допустимыми html-тегами являются:
<b> - жирный текст
<i> - курсив
<s> - зачеркнутый текст
<u> - подчеркнутый текст
<a> - ссылка
<code> - Моноширинный текст
<pre> - мультистрочный моноширинный
<blockquote> - блок цитаты

Все остальные теги являются недопустимыми и должны быть удалены.
Также если ты встречаешь markdown-разметку, например **, то её стоит заменить на html-теги <b> и </b>.
В ответ просто пришли исправленный текст, никаких комментариев и пояснений.
"""
system_prompt

In [ ]:
from __future__ import annotations
from yandex_cloud_ml_sdk import YCloudML

messages = [
    {
        "role": "system",
        "text": system_prompt,
    },
    {
        "role": "user",
        "text": f"Ответ LLM, который нужно исправить:\n{perplex_ans}",
    },
]

In [ ]:
model = sdk.models.completions("yandexgpt").configure(temperature=0.5)
model

In [ ]:
result = model.run_deferred(messages).wait()

In [ ]:
for alternative in result:
    print(alternative)


In [ ]:
result.alternatives[0].text

In [ ]:
alternative.text

## VLM

In [ ]:
sdk.models.completions("qwen2.5-vl")

In [ ]:
dir(sdk.models)